In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 4)

START = "2016-01-01"
END   = None  # usa hoy

# MVP: 10 emisoras MX (puedes cambiarlas luego)
TICKERS = [
    "WALMEX.MX","FEMSAUBD.MX","GMEXICOB.MX","GFINBURO.MX",
    "BIMBOA.MX","CEMEXCPO.MX","ALFAA.MX","KIMBERA.MX","PE&OLES.MX"
]

# Mercado: IPC (en Yahoo suele ser ^MXX)
MARKET = "^MXX"

# Macro proxy (MVP): USD/MXN y 13-week T-bill (proxy de tasa global)
FX = "MXN=X"
RATE_PROXY = "^IRX"


In [ ]:
def get_adj_close(tickers, start=START, end=END):
    px = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)["Close"]
    if isinstance(px, pd.Series):
        px = px.to_frame()
    px = px.dropna(how="all")
    return px

# Acciones + mercado
px_stocks = get_adj_close(TICKERS, START, END)
px_mkt    = get_adj_close([MARKET], START, END).rename(columns={MARKET:"MKT"})

# Macros (MVP)
px_fx   = get_adj_close([FX], START, END).rename(columns={FX:"USDMXN"})
px_rate = get_adj_close([RATE_PROXY], START, END).rename(columns={RATE_PROXY:"RATE"})

# Resample a fin de mes
m_stocks = px_stocks.resample("ME").last()
m_mkt    = px_mkt.resample("ME").last()
m_fx     = px_fx.resample("ME").last()
m_rate   = px_rate.resample("ME").last()

# Returns (log) para acciones y mercado
r_stocks = np.log(m_stocks / m_stocks.shift(1))
r_mkt    = np.log(m_mkt / m_mkt.shift(1))

# Cambios macro (diferencias) – simple para MVP
d_fx   = np.log(m_fx / m_fx.shift(1))          # % cambio FX
d_rate = m_rate.diff() / 100.0                 # cambio en tasa proxy (escala)

data = r_stocks.join(r_mkt).join(d_fx).join(d_rate).dropna()
data.tail()


In [ ]:
results = []

X_base = data[["MKT","USDMXN","RATE"]]
X_base = sm.add_constant(X_base)

for ticker in TICKERS:
    y = data[ticker]
    df = pd.concat([y, X_base], axis=1).dropna()
    y2 = df[ticker]
    X2 = df.drop(columns=[ticker])

    model = sm.OLS(y2, X2).fit()
    b_fx   = model.params.get("USDMXN", np.nan)
    b_rate = model.params.get("RATE", np.nan)

    results.append({
        "ticker": ticker,
        "beta_fx": b_fx,
        "beta_rate": b_rate,
        "r2": model.rsquared,
        "n_obs": int(model.nobs)
    })

betas = pd.DataFrame(results).set_index("ticker")

# Sensibilidad típica = |beta| * volatilidad de la macro
sig_fx   = data["USDMXN"].std()
sig_rate = data["RATE"].std()

betas["S_fx"]   = betas["beta_fx"].abs() * sig_fx
betas["S_rate"] = betas["beta_rate"].abs() * sig_rate

# Índice agregado (pesos iguales en MVP)
betas["MSI_raw"] = betas["S_fx"] + betas["S_rate"]

# Normaliza 0-100
betas["MSI_0_100"] = 100 * (betas["MSI_raw"] - betas["MSI_raw"].min()) / (betas["MSI_raw"].max() - betas["MSI_raw"].min())
betas = betas.sort_values("MSI_0_100", ascending=False)

betas


In [ ]:
# 1) Define shocks (1σ) con los datos que ya tienes
shock_fx = data["USDMXN"].std()   # 1σ del cambio mensual en FX (log)
shock_rate = data["RATE"].std()   # 1σ del cambio mensual en RATE (tu proxy)

# 2) Impacto esperado (aprox en %)
betas["impact_fx_1sd_pct"]   = 100 * (betas["beta_fx"]   * shock_fx)
betas["impact_rate_1sd_pct"] = 100 * (betas["beta_rate"] * shock_rate)

# 3) Magnitud total (para ranking por "susceptibilidad" en impactos)
betas["impact_total_abs_pct"] = betas["impact_fx_1sd_pct"].abs() + betas["impact_rate_1sd_pct"].abs()

# 4) Tabla final ordenada por mayor impacto total (más "susceptibles")
impact_table = betas[[
    "impact_rate_1sd_pct", "impact_fx_1sd_pct", "impact_total_abs_pct",
    "beta_rate", "beta_fx", "r2", "n_obs"
]].sort_values("impact_total_abs_pct", ascending=False)

impact_table.head(15)


In [ ]:
ax = betas["MSI_0_100"].sort_values().plot(kind="barh")
ax.set_title("MSI-MX (MVP) - Sensibilidad a FX y RATE proxy")
ax.set_xlabel("MSI (0-100)")
plt.show()


In [ ]:
tickers_ok = [c for c in data.columns if c not in ["MKT","USDMXN","RATE"]]
tickers_ok



In [ ]:
import os
os.makedirs("reports", exist_ok=True)

betas.to_csv("reports/msi_mx_mvp_ranking.csv")
print("Guardado en reports/msi_mx_mvp_ranking.csv")
betas.head(10)


In [ ]:
WINDOW = 36  # meses

def compute_msi_window(w: pd.DataFrame, tickers: list[str]) -> pd.DataFrame:
    X = sm.add_constant(w[["MKT","USDMXN","RATE"]])
    sig_fx, sig_rate = w["USDMXN"].std(), w["RATE"].std()

    rows = []
    for t in tickers:
        y = w[t]
        df = pd.concat([y, X], axis=1).dropna()
        if len(df) < max(24, WINDOW-3):  # mínimo de observaciones
            continue

        y2 = df[t]
        X2 = df.drop(columns=[t])
        m = sm.OLS(y2, X2).fit()

        b_fx   = m.params.get("USDMXN", np.nan)
        b_rate = m.params.get("RATE", np.nan)

        S_fx   = abs(b_fx) * sig_fx
        S_rate = abs(b_rate) * sig_rate

        rows.append({
            "ticker": t,
            "beta_fx": b_fx,
            "beta_rate": b_rate,
            "S_fx": S_fx,
            "S_rate": S_rate,
            "MSI_raw": S_fx + S_rate,
            "r2": m.rsquared,
            "n_obs": int(m.nobs),
        })

    out = pd.DataFrame(rows).set_index("ticker")
    out["MSI_0_100"] = 100 * (out["MSI_raw"] - out["MSI_raw"].min()) / (out["MSI_raw"].max() - out["MSI_raw"].min())
    return out.sort_values("MSI_0_100", ascending=False)

# construir panel rolling
msi_list = []
for end_i in range(WINDOW, len(data) + 1):
    w = data.iloc[end_i-WINDOW:end_i].copy()
    msi_w = compute_msi_window(w, tickers_ok)
    msi_w["date"] = w.index[-1]
    msi_list.append(msi_w.reset_index())

msi_panel = pd.concat(msi_list, ignore_index=True)
msi_panel.tail()


In [ ]:
top5 = (msi_panel[msi_panel["date"] == msi_panel["date"].max()]
        .sort_values("MSI_0_100", ascending=False)
        .head(5)["ticker"]
        .tolist())

pivot = (msi_panel[msi_panel["ticker"].isin(top5)]
         .pivot(index="date", columns="ticker", values="MSI_0_100")
         .sort_index())

pivot.plot(title="MSI-MX Rolling (Top 5 más sensibles)")
plt.ylabel("MSI (0-100)")
plt.show()


In [ ]:
msi_panel.to_csv("reports/msi_mx_rolling_panel.csv", index=False)
print("Guardado en reports/msi_mx_rolling_panel.csv")


In [ ]:
top10 = impact_table.head(10).copy()
top10[["impact_rate_1sd_pct","impact_fx_1sd_pct"]].plot(kind="bar", title="Impacto esperado ante shocks 1σ (Top 10)")
plt.ylabel("Impacto esperado (%)")
plt.show()


In [ ]:
# --- Resumen en texto del Top 10 (impactos 1σ) ---
top10 = impact_table.head(10).copy()

lines = []
lines.append("Resumen (Top 10) — Impacto esperado ante shocks 1σ")
lines.append("Interpretación: signo (+) sube, (-) baja. Valores ~ % de retorno mensual.\n")

for t, row in top10.iterrows():
    ir = row["impact_rate_1sd_pct"]
    fx = row["impact_fx_1sd_pct"]

    dom = "TASAS" if abs(ir) > abs(fx) else "FX"
    dir_rate = "sube" if ir > 0 else "baja"
    dir_fx   = "sube" if fx > 0 else "baja"

    lines.append(
        f"- {t}: "
        f"Tasas(1σ) → {ir:+.2f}% ({dir_rate}), "
        f"FX(1σ) → {fx:+.2f}% ({dir_fx}). "
        f"Dominante: {dom}."
    )

print("\n".join(lines))


In [ ]:
with open("reports/top10_impact_summary.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))
print("Guardado en reports/top10_impact_summary.txt")
